In [115]:
class Lexer:
    def __init__(self):
        self.string = ""
        self.pointer = 0
        self.keywords = {
            "int": (5, "-"),
            "else": (15, "-"),
            "if": (17, "-"),
            "while": (20, "-")
        }

        self.constants = {
            "整数": (100, "整数")
        }

        self.operators = {
            "+": (41, "-"),
            "-": (42, "-"),
            "*": (43, "-"),
            "/": (44, "-"),
            "%": (45, "-"),
            "=": (46, "-"),
            ">": (47, "-"),
            ">=": (48, "-"),
            "<": (49, "-"),
            "<=": (50, "-"),
            "==": (51, "-"),
            "!=": (52, "-"),
            "&&": (53, "-"),
            "||": (54, "-"),
            "!": (55, "-"),
            "++": (56, "-"),
            "--": (57, "-")
        }

        self.delimiters = {
            "(": (81, "-"),
            ")": (82, "-"),
            ",": (83, "-"),
            ";": (84, "-"),
            "{": (86, "-"),
            "}": (87, "-"),
            "[": (88, "-"),
            "]": (89, "-")
        }
    def get_operator_symbol(self, op):
        for symbol, (token_type, _) in self.operators.items():
            if token_type == op:
                return symbol
        return None
    def count_tokens(self):
        tokens = []
        current_token = ""
        i = 0
        while i < len(self.string):
            if self.string[i].isdigit(): 
                current_token += self.string[i]
                i += 1
                while i < len(self.string) and self.string[i].isdigit():
                    current_token += self.string[i]
                    i += 1
                tokens.append((100, current_token))  
                current_token = ""
            elif self.string[i].isalnum() or self.string[i] == '_':
                current_token += self.string[i]
                i += 1
                while i < len(self.string) and (self.string[i].isalnum() or self.string[i] == '_'):
                    current_token += self.string[i]
                    i += 1
                if current_token in self.keywords:
                    tokens.append(self.keywords[current_token])
                else:
                    tokens.append(self.is_identifier(current_token))
                current_token = ""
            elif self.string[i] in self.operators or self.string[i:i+2] in self.operators:
                if self.string[i:i+2] in self.operators:
                    tokens.append(self.operators[self.string[i:i+2]])
                    i += 2
                else:
                    tokens.append(self.operators[self.string[i]])
                    i += 1
            elif self.string[i] in self.delimiters:
                tokens.append(self.delimiters[self.string[i]])
                i += 1
            elif self.string[i].isspace():
                i += 1
            else:
                raise ValueError("Illegal character: {}".format(self.string[i]))
        return len(tokens)
    
    def is_identifier(self, token):
        return 111, token

    def get_next_token(self):
        char = self.string[self.pointer]
        token=""
        if char.isdigit(): 
            token = char
            self.pointer += 1
            while self.pointer < len(self.string) and self.string[self.pointer].isdigit():
                token += self.string[self.pointer]
                self.pointer += 1
            return (100, token)  
        elif char.isalnum() or char == '_':
            token = char
            self.pointer += 1
            while self.pointer < len(self.string) and (self.string[self.pointer].isalnum() or self.string[self.pointer] == '_'):
                token += self.string[self.pointer]
                self.pointer += 1
            if token in self.keywords:
                return self.keywords[token]
            else:
                return self.is_identifier(token)
        elif char in self.operators or (self.string[self.pointer:self.pointer + 2]) in self.operators:
            if (self.string[self.pointer:self.pointer + 2]) in self.operators:
                self.pointer += 2
                return self.operators[self.string[self.pointer-2:self.pointer]]
            else:
                self.pointer += 1
                return self.operators[char]
        elif char in self.delimiters:
            self.pointer += 1
            return self.delimiters[char]
        elif char.isspace():
            self.pointer += 1
            return self.get_next_token()

    def analyze(self):
        if self.pointer >= len(self.string):
            return None
        return self.get_next_token()

    def analyze_file(self, file_path):
        try:
            with open(file_path, 'r') as file:
                file_content = file.read()  
                self.string = file_content  
                self.token_num = self.count_tokens()
        except FileNotFoundError:
            print("File not found:", file_path)


In [116]:
class SymbolTableEntry:
    def __init__(self, name, ndim=0, place=None, array=None, offset=None):
        self.name = name          
        self.ndim = ndim         
        self.place = place        
        self.array = array        
        self.offset = offset      
        self.in_place = []      

    def set_dimensions(self, ndim):
        self.ndim = ndim

    def add_in_place(self, value):
        self.in_place.append(value)

    def __repr__(self):
        return f"SymbolTableEntry(name={self.name}, ndim={self.ndim}, place={self.place}, array={self.array}, offset={self.offset}, in_place={self.in_place})"
    
class SymbolTable:
    def __init__(self):
        self.table = {}

    def add_entry(self, name, ndim=0, place=None, array=None, offset=None):
        entry = SymbolTableEntry(name, ndim, place, array, offset)
        self.table[name] = entry
        return entry

    def get_entry(self, name):
        return self.table.get(name, None)

    def __repr__(self):
        return "\n".join(str(entry) for entry in self.table.values())


In [117]:
class SymbolTableEntry:
    def __init__(self, name, ndim=0, place=None, array=None, offset=None):
        self.name = name          
        self.ndim = ndim         
        self.place = place        
        self.array = array        
        self.offset = offset      
        self.in_place = []        

    def set_dimensions(self, ndim):
        self.ndim = ndim

    def add_in_place(self, value):
        self.in_place.append(value)

    def __repr__(self):
        return f"SymbolTableEntry(name={self.name}, ndim={self.ndim}, place={self.place}, array={self.array}, offset={self.offset}, in_place={self.in_place})"


def makelist(nextquad):
    return [nextquad]


class SymbolTable:
    def __init__(self):
        self.table = {}

    def add_entry(self, name, ndim=0, place=None, array=None, offset=None):
        entry = SymbolTableEntry(name, ndim, place, array, offset)
        self.table[name] = entry
        return entry

    def get_entry(self, name):
        return self.table.get(name, None)

    def __repr__(self):
        return "\n".join(str(entry) for entry in self.table.values())


class Parser:
    def __init__(self, lexer):
        self.lexer = lexer
        self.current_token = None
        self.step = 1
        self.temp_count = 0
        self.quadruples = []
        self.symbol_table = SymbolTable()
        self.quad_index = 1  
        self.next_quad = 0
    def nextquad(self):
        return self.next_quad

    def makelist(self, index=None):
        if index is not None:
            return [index]
        else:
            return []

    def backpatch(self, list_, target):
        if list_ is None or list_ == []:
            self.quadruples[0][-1] = target
        else:
            for index in list_:
                self.quadruples[index][-1] = target

    def merge(self, list1, list2):
        if not list2:
            return list1
        elif not list1:
            return list2
        else:
            combined_list = list2 + list1
            return combined_list

    def eat(self, token_type):
        if self.current_token is not None and self.current_token[0] == token_type:
            self.current_token = self.lexer.analyze()

    def print_step(self, production):
        print(f"({self.step}) {production}")
        self.step += 1

    def emit(self, op, arg1, arg2, result):
        self.next_quad += 1
        self.quadruples.append([op, arg1, arg2, result])

    def newtemp(self):
        self.temp_count += 1
        return f"t{self.temp_count}"

    def factor(self):
        if self.current_token is None:
            return None
        token_type = self.current_token[0]
        if token_type == 100:
            self.print_step("factor ⟶ num")
            token_value = self.current_token[1]
            self.eat(100)
            return token_value
        elif token_type == 111:
            self.print_step("factor ⟶ loc")
            loc_place, loc_offset = self.loc()
            if loc_offset is None:
                return loc_place
            else:
                factor_place = self.newtemp()
                self.emit('=[]', f"{loc_place}[{loc_offset}]", '-', factor_place)
                return factor_place
        elif token_type == 81:  
            self.print_step("factor ⟶ (expr)")
            self.eat(81)  
            result = self.expression()
            self.eat(82)  
            return result
        else:
            raise SyntaxError(f"Unexpected token: {self.current_token}")

    def loc(self):
        if self.current_token is None:
            return None, None
        self.print_step("loc ⟶ id resta")
        id_value = self.current_token[1]
        self.eat(111)  
        entry = self.symbol_table.get_entry(id_value)
        if entry is None:
            entry = self.symbol_table.add_entry(id_value)
        inArray = id_value
        place, offset = self.resta(inArray)
        entry.place = place
        entry.offset = offset
        return place, offset

    def resta(self, inArray):
        if self.current_token is None:
            self.print_step("resta ⟶ ℇ")
            return inArray, None
        if self.current_token[0] == 88:
            self.print_step("resta ⟶ [elist]")
            self.eat(88)  
            array = self.symbol_table.get_entry(inArray)
            array.place = inArray
            self.elist(array)
            self.eat(89)  
            place = self.newtemp()
            self.emit('-', array.place, 'C', place)
            offset = self.newtemp()
            self.emit('*', array.offset, 'w', offset)
            return place, offset
        else:
            self.print_step("resta ⟶ ℇ")
            return inArray, None

    def elist(self, array):
        if self.current_token is None:
            return None
        self.print_step("elist ⟶ expr rest1")
        expr_place = self.expression()
        array.add_in_place(expr_place)
        rest1_inPlace = expr_place
        rest1_inNdim = 1
        array.offset = self.rest1(array, rest1_inNdim, rest1_inPlace)
        return None

    def rest1(self, array, inNdim, inPlace):
        if self.current_token is None:
            self.print_step("rest1 ⟶ ℇ")
            return inPlace
        if self.current_token[0] == 83:
            self.print_step("rest1 ⟶ , expr rest11")
            self.eat(83)  
            expr_place = self.expression()
            t = self.newtemp()
            m = inNdim + 1
            self.emit('*', inPlace, f"n{m}", t)
            self.emit('+', t, expr_place, t)
            rest11_inPlace = t
            rest11_inNdim = m
            return self.rest1(array, rest11_inNdim, rest11_inPlace)
        else:
            self.print_step("rest1 ⟶ ℇ")
            return inPlace

    def term(self):
        if self.current_token is None:
            return None
        self.print_step("term ⟶ factor")
        left = self.factor()
        while self.current_token is not None and self.current_token[0] in (43, 44):  
            self.eat(82)  
            op = self.current_token[0]
            self.eat(op)
            right = self.factor()
            temp = self.newtemp()
            self.emit(self.lexer.get_operator_symbol(op), left, right, temp)
            left = temp
        return left

    def expression(self):
        if self.current_token is None:
            return None
        self.print_step("expr ⟶ term")
        left = self.term()
        while self.current_token is not None and self.current_token[0] in (41, 42):  
            op = self.current_token[0]
            self.eat(op)
            right = self.term()
            temp = self.newtemp()
            self.emit(self.lexer.get_operator_symbol(op), left, right, temp)
            left = temp
        return left

    def bool_expr(self):
        if self.current_token is None:
            return None
        self.print_step("bool ⟶ equality")
        return self.equality()

    def equality(self):
        if self.current_token is None:
            return None, None
        self.print_step("equality ⟶ rel rest4")
        rel_truelist, rel_falselist = self.rel()
        return self.rest4(rel_truelist, rel_falselist)
    def rel(self):
        if self.current_token is None:
            return None, None
        self.print_step("rel ⟶ expr rop_expr")
        expr_place = self.expression()
        rop_expr_inPlace = expr_place
        rop_expr_truelist, rop_expr_falselist = self.rop_expr(rop_expr_inPlace)
        rel_truelist = rop_expr_truelist
        rel_falselist = rop_expr_falselist
        return rel_truelist, rel_falselist

    def rop_expr(self, inPlace):
        if self.current_token is None:
            return None, None
        if self.current_token[0] in (47, 48, 49, 50):
            self.print_step("rop_expr ⟶ rel_op expr")
            rel_op = self.current_token[0]
            self.eat(rel_op)
            expr_place = self.expression()
            truelist = self.makelist(self.nextquad())
            falselist = self.makelist(self.nextquad() + 1)
            self.emit(f'j{self.lexer.get_operator_symbol(rel_op)}', inPlace, expr_place, '-')
            self.emit('j', '-', '-', '-')
            return truelist, falselist
        elif self.current_token[0] == 82:  
            self.print_step("rop_expr ⟶ ℇ")
            nextquad = self.nextquad()  
            truelist = self.makelist(nextquad)
            nextquad_plus_one = self.nextquad()+1  
            falselist = self.makelist(nextquad_plus_one)
            self.emit('jnz', inPlace, '-', '-')
            self.emit('j', '-', '-', '-')
            return truelist, falselist
        else:
            raise SyntaxError(f"Unexpected token: {self.current_token}")

    def rest4(self, inTruelist, inFalselist):
        if self.current_token is None:
            return None, None
        if self.current_token[0] in (51, 52):  
            self.print_step("rest4 ⟶ == rel rest41 | != rel rest41")
            self.rel()
            return None, None
        elif self.current_token[0] == 82:  
            self.print_step("rest4 ⟶ ℇ")
            return inTruelist, inFalselist
        else:
            raise SyntaxError(f"Unexpected token: {self.current_token}")

    def stmts(self):
        if self.current_token is None:
            return self.makelist()
        self.print_step("stmts ⟶ stmt rest0")
        stmt_nextlist = self.stmt()
        self.rest0(stmt_nextlist)

    def stmt(self):
        if self.current_token is None:
            return self.makelist()
        if self.current_token[0] == 111:  
            self.print_step("stmt ⟶ loc = expr;")
            loc_place, loc_offset = self.loc()
            self.eat(46)  
            expr_place = self.expression()
            if loc_offset is None:
                self.emit('=', expr_place, '-', loc_place)
            else:
                self.emit('[]=', expr_place, '-', f"{loc_place}[{loc_offset}]")
            self.eat(84) 
            return self.makelist()
        elif self.current_token[0] == 17:  
            self.print_step("stmt ⟶ if (bool) stmt else stmt")
            self.eat(17)  
            self.eat(81)  
            bool_truelist, bool_falselist = self.bool_expr()
            self.eat(82)  
            m1 = self.nextquad()
            stmt1_nextlist = self.stmt()
            n_nextlist = self.makelist(self.nextquad())
            self.emit('j', '-', '-', 0)
            self.eat(15)  
            m2 = self.nextquad()
            stmt2_nextlist = self.stmt()
            self.backpatch(bool_truelist, m1)
            self.backpatch(bool_falselist, m2)
            return self.merge(stmt1_nextlist,self.merge(n_nextlist,stmt2_nextlist) )
        elif self.current_token[0] == 20:  
            self.print_step("stmt ⟶ while (bool) stmt")
            self.eat(20)  
            self.eat(81)  
            m1 = self.nextquad()
            bool_truelist, bool_falselist = self.bool_expr()
            self.eat(82)  
            m2 = self.nextquad()
            stmt1_nextlist = self.stmt()
            self.backpatch(stmt1_nextlist, m1)
            self.backpatch(bool_truelist, m2)
            self.emit('j', '-', '-', m1)
            return bool_falselist
        else:
            raise SyntaxError(f"Unexpected token: {self.current_token}")

    def rest0(self, inNextlist=None):
        if self.current_token is None:
            self.print_step("rest0 ⟶ ℇ")
            return inNextlist
        
        if self.current_token[0] in (111, 17, 20):  
            self.print_step("rest0 ⟶ stmt rest0")
            m_quad = self.nextquad()
            self.stmt()
            self.backpatch(inNextlist, m_quad)
            stmt_nextlist = self.stmt()
            rest01_nextlist = self.rest0(stmt_nextlist)
            return rest01_nextlist
        else:
            self.print_step("rest0 ⟶ ℇ")
            return inNextlist


    def parse(self):
        self.current_token = self.lexer.analyze()
        self.stmts()
        return self.quadruples

lexer = Lexer()  
lexer.analyze_file("source_code10.txt")
parser = Parser(lexer)
quadruples = parser.parse()

for idx, quad in enumerate(quadruples):
    print(f"{idx}: {quad}")


(1) stmts ⟶ stmt rest0
(2) stmt ⟶ if (bool) stmt else stmt
(3) bool ⟶ equality
(4) equality ⟶ rel rest4
(5) rel ⟶ expr rop_expr
(6) expr ⟶ term
(7) term ⟶ factor
(8) factor ⟶ loc
(9) loc ⟶ id resta
(10) resta ⟶ ℇ
(11) rop_expr ⟶ rel_op expr
(12) expr ⟶ term
(13) term ⟶ factor
(14) factor ⟶ loc
(15) loc ⟶ id resta
(16) resta ⟶ ℇ
(17) rest4 ⟶ ℇ
(18) stmt ⟶ if (bool) stmt else stmt
(19) bool ⟶ equality
(20) equality ⟶ rel rest4
(21) rel ⟶ expr rop_expr
(22) expr ⟶ term
(23) term ⟶ factor
(24) factor ⟶ loc
(25) loc ⟶ id resta
(26) resta ⟶ ℇ
(27) rop_expr ⟶ ℇ
(28) rest4 ⟶ ℇ
(29) stmt ⟶ loc = expr;
(30) loc ⟶ id resta
(31) resta ⟶ ℇ
(32) expr ⟶ term
(33) term ⟶ factor
(34) factor ⟶ loc
(35) loc ⟶ id resta
(36) resta ⟶ ℇ
(37) term ⟶ factor
(38) factor ⟶ loc
(39) loc ⟶ id resta
(40) resta ⟶ ℇ
(41) stmt ⟶ loc = expr;
(42) loc ⟶ id resta
(43) resta ⟶ ℇ
(44) expr ⟶ term
(45) term ⟶ factor
(46) factor ⟶ loc
(47) loc ⟶ id resta
(48) resta ⟶ ℇ
(49) stmt ⟶ if (bool) stmt else stmt
(50) bool ⟶ equalit